# Redes Neuronales
## Preparacion de datos para el modelo

In [60]:
import sys
sys.path.append('../')

import Utilidades as ut
import Modelos as md
import numpy as np
import pandas as pd


df_train = pd.read_pickle("../Archivos/Neuronales_entrenamiento.pkl")
df_test = pd.read_pickle("../Archivos/Neuronales_validacion.pkl")

x_train, y_train = ut.split_labels(df_train)
x_test, y_test = ut.split_labels(df_test)

#Convertimos las fechas a numeros (cantidad de dias transcurridos) y luego las normalizamos
x_train, x_test = ut.conversion_fechas(x_train, x_test)
x_train, x_test = ut.codificar_categoricas(x_train, y_train, x_test, modo='catboost')
x_train, x_test = ut.normalizacion_numericas(x_train, x_test, modo='normalizacion')
x_train.head(3)

/home/julian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Sales_Contract_No,Opportunity_Name,Planned_Opportunity_Duration,Account_Name,Product_Name,Planned_Time_Until_Deliver,Bureaucratic_Code,Last_Modified_By,Opportunity_Owner,Product_Family,Opportunity_Type,Total_Amount(USD),Account_Owner,Account_Type,Opportunity_Created_Date,Territory,Product_Amount_Deviation_of_Product_Family_rate,Planned_Deliver_Duration,Account_Created_Date,Billing_Country
0,0.42,-0.08,13.14,-0.02,0.15,15.21,0.14,0.37,0.38,0.34,0.50,0.89,0.33,0.39,-4.78,0.36,0.97,-0.41,-1.35,0.53
1,-0.39,-0.08,5.51,-0.02,0.15,6.19,0.14,0.37,0.38,0.34,0.50,1.63,0.33,-1.09,-4.22,0.36,2.44,0.28,-0.96,0.53
2,-0.66,-0.08,5.98,-0.02,0.15,6.75,0.14,0.37,0.38,0.34,-1.06,0.47,0.33,0.39,-4.19,0.36,-0.70,0.26,-1.29,0.53


## Creacion del modelo.

In [61]:
import keras
import tensorflow as tf
#from keras import backend
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import Dropout
#from keras.regularizers import l1
#from keras.regularizers import l2
#from keras.regularizers import l1_l2


x_train_vector = ut.df_a_vector(x_train)
y_train_vector = ut.df_a_vector(y_train)
x_test_vector = ut.df_a_vector(x_test)
y_test_vector = ut.df_a_vector(y_test)

#input_dim = x_train.shape[1]
#
#alfa = 1e-3
#
#model = Sequential()
#model.add(Dense(64, input_dim=input_dim, activation='relu'))
#model.add(Dropout(0.25))
#model.add(Dense(128, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='relu'))
##model.add(Dropout(0.25))
##model.add(Dense(256, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='tanh'))
#model.add(Dropout(0.5))
#model.add(Dense(128, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(64, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='tanh'))
#model.add(Dropout(0.25))
#model.add(Dense(16, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='relu'))
#model.add(Dense(8, kernel_regularizer=l1(alfa), bias_regularizer=l1(alfa), activation='relu'))
#model.add(Dense(1, activation='tanh'))
#
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#
#backend.set_value(model.optimizer.learning_rate, 5e-4)

best_hparams = ut.hyperparams_from_json('../Archivos/Neuronales')
model = md.get_neural_network_model(best_hparams, x_train.shape[1])

epochs = 80#300
batch_size = 256


my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint("Neuronales_Mejor_Modelo.hdf5", 
                                       monitor='val_loss', 
                                       verbose=0,
                                       save_best_only=True, 
                                       mode='min'),
    
    tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                     min_delta=0.001,#0.0001,
                                     mode='min',
                                     patience=30),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                      mode='min',
                                      factor=0.5,
                                      patience=5,
                                      cooldown=0, 
                                      min_lr=1e-24)
]

fit_dict = {
    #'x' : x_train_vector,
    #'y' : y_train_vector,
    #'validation_data' : (x_test_vector, y_test_vector),
    'epochs' : epochs,
    'batch_size' : batch_size,
    'verbose' : 1,
    'callbacks' : my_callbacks
}


history = model.fit(x_train_vector,
                    y_train_vector,
                    validation_data=(x_test_vector, y_test_vector),
                    **fit_dict)


#Mejor modelo hasta el momento val_loss = 0.3858

Cargando hiperparametros desde el archivo: '../Archivos/Neuronales_best_hyperparam.json'
Epoch 1/80
48/48 [==============================] - 2s 49ms/step - loss: 15.4422 - val_loss: 3.1896 - lr: 0.0042
Epoch 2/80
48/48 [==============================] - 2s 35ms/step - loss: 1.5944 - val_loss: 0.9923 - lr: 0.0042
Epoch 3/80
48/48 [==============================] - 1s 29ms/step - loss: 0.8092 - val_loss: 0.7797 - lr: 0.0042
Epoch 4/80
48/48 [==============================] - 1s 22ms/step - loss: 0.7282 - val_loss: 0.8017 - lr: 0.0042
Epoch 5/80
48/48 [==============================] - 1s 22ms/step - loss: 0.7032 - val_loss: 0.8222 - lr: 0.0042
Epoch 6/80
48/48 [==============================] - 1s 21ms/step - loss: 0.7131 - val_loss: 0.7290 - lr: 0.0042
Epoch 7/80
48/48 [==============================] - 1s 21ms/step - loss: 0.6819 - val_loss: 0.8161 - lr: 0.0042
Epoch 8/80
48/48 [==============================] - 1s 21ms/step - loss: 0.6849 - val_loss: 0.7797 - lr: 0.0042
Epoch 9/80
48/

## Resultados

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['val_loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Accuracy vs Epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#new_model = keras.models.load_model('Neuronales_Mejor_Modelo.hdf5')
#new_predictions = new_model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = [a > 0.5 for a in model.predict(x_test_vector).flatten()]

score = accuracy_score(y_pred, y_test_vector)
score

In [ ]:
model.evaluate(x_test_vector, y_test_vector)#, metrics=['accuracy'])

In [ ]:
#Podemos analizar las filas que originaron los errores de prediccion

analysis_df = y_test.copy()
analysis_df['Predicted'] = pd.Series(y_pred).astype(int)
differences = (analysis_df['Predicted'] != analysis_df['Stage'])
df_test[differences]